In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.models import load_model
from keras.preprocessing import image
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import random
import os


In [1]:
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'
test_dir = 'dataset/test'

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Veri kümelerini oluşturma
batch_size = 20

train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_dataset = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 14589 images belonging to 12 classes.
Found 1800 images belonging to 12 classes.
Found 1200 images belonging to 12 classes.


In [ ]:

# ResNet-50 modeli
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


model_resnet = Sequential()
model_resnet.add(resnet_model)
model_resnet.add(Flatten())
model_resnet.add(Dense(512, activation='relu'))
model_resnet.add(Dense(12, activation='softmax'))

# Modeli derleme
model_resnet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Eğitim verilerini ve doğrulama verilerini belirtme ve eğitim
num_epochs = 35
#history_vgg = model_vgg.fit(train_dataset, epochs=num_epochs, validation_data=validation_dataset)
history_resnet = model_resnet.fit(train_dataset, epochs=num_epochs, validation_data=validation_dataset)


In [5]:
# ResNet modelini kaydetme
model_resnet.save("resnet_model_35epoch.h5")


C:\Users\koktu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
model_path = 'resnet_model_35epoch.h5'
model = load_model(model_path)

In [ ]:


selected_model_resnet = model

#Test veri setinin bulunduğu klasörü belirtin
test_images_dir = test_dir  # Test görüntülerinin bulunduğu klasörün yolu


#Sınıf etiketlerini alın
class_labels = list(train_dataset.class_indices.keys())


#Tüm test görüntülerini dolaşma

correct_predictions = 0
total_predictions = 0

for root, dir, files in os.walk(test_images_dir):
    for img_file in files:
        # Görüntü dosyasının tam yolunu oluşturun
        img_path = os.path.join(root, img_file)

        # Görüntüyü modele uygun formata getirme
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Veriyi normalize etme

        # Tahminlerde bulunma
    
        predictions_resnet = selected_model_resnet.predict(img_array)

            # Tahmin edilen sınıf indekslerini alın
            
        predicted_class_resnet = np.argmax(predictions_resnet)

        # Orijinal sınıfı alın
        true_class = img_path.split(os.path.sep)[-2]  # Örneğin, 'dataset/test/class1/image.jpg' ise, sınıf 'class1' olacak

            # Tahmin edilen sınıf adlarını alın
        
        predicted_class_label_resnet = class_labels[predicted_class_resnet]
        
        # plt.imshow(img)
        # plt.title(f'Orijinal Sınıf: {true_class} Predicted Class: {predicted_class_label_resnet}')
        # plt.show()

        if true_class == class_labels[predicted_class_resnet]:
                correct_predictions += 1

        total_predictions += 1
            
        print(f'Görüntü: {img_file}, Orijinal Sınıf: {true_class}, ResNet Tahmini: {predicted_class_label_resnet}')
        